# Define the documents 

In [1]:

doc_trump = "Mr. Trump became president after winning the political election. Though h e lost the support of some republican friends, Trump is friends with President Putin"  
doc_election = "President Trump says Putin had no political interference is the electi on outcome. He says it was a witchhunt by political parties. He claimed President Puti n is a friend who had nothing to do with the election"  
doc_putin = "Post elections, Vladimir Putin became President of Russia. President Puti n had served as the Prime Minister earlier in his political career"  
documents = [doc_trump, doc_election, doc_putin] 

# Scikit Learn 

In [2]:
import sklearn
from sklearn.feature_extraction.text  import CountVectorizer 
import pandas as pd  


# Create the Document Term Matrix 

In [3]:

count_vectorizer = CountVectorizer(stop_words='english') 
count_vectorizer = CountVectorizer() 
sparse_matrix = count_vectorizer.fit_transform(documents)  


# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word freq uencies. 

In [4]:

doc_term_matrix = sparse_matrix.todense() 
df = pd.DataFrame(doc_term_matrix,columns=count_vectorizer.get_feature_names(),index=['doc_trump', 'doc_election', 'doc_putin']) 
df 

,after,as,became,by,career,claimed,do,earlier,electi,election,...,the,though,to,trump,vladimir,was,who,winning,witchhunt,with
doc_trump,1,0,1,0,0,0,0,0,0,1,...,2,1,0,2,0,0,0,1,0,1
doc_election,0,0,0,1,0,1,1,0,1,1,...,2,0,1,1,0,1,1,0,1,1
doc_putin,0,1,1,0,1,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


# Compute Cosine Similarity 


In [5]:
from sklearn.metrics.pairwise import cosine_similarity 
print(cosine_similarity(df, df))  

[[1.         0.4361843  0.37450294]
 [0.4361843  1.         0.35745328]
 [0.37450294 0.35745328 1.        ]]


without library

In [6]:
df_transpose=df.T

In [7]:
lis1=df_transpose.doc_trump.tolist()
lis2=df_transpose.doc_election.tolist()
lis3=df_transpose.doc_putin.tolist()

In [8]:
import math
from math import sqrt

def cosine_sim(list1,list2):      #list are list of count of different words
    summ=0
    v1=0
    v2=0
    for i,j in zip(list1,list2):
        summ+=(i*j)
        v1+=(i*i)
        v2+=(j*j)
    cossim=summ/(sqrt(v1)*sqrt(v2))
    return cossim

In [9]:
cosine_sim(lis1,lis3)

0.3745029431365692

 we are getting the similarity between doc_trump and doc_putin,which is exactly similar to the sklearn library

# soft cosine similarity

In [10]:
# Define the documents 
doc_soup = "Soup is a primarily liquid food, generally served warm or hot (but may be cool or cold), that is made by combining ingredients of meat or vegetables with stoc k, juice, water, or another liquid. "  
doc_noodles = "Noodles are a staple food in many cultures. They are made from unleaven ed dough which is stretched, extruded, or rolled flat and cut into one of a variety of shapes."  
doc_dosa = "Dosa is a type of pancake from the Indian subcontinent, made from a fermen ted batter. It is somewhat similar to a crepe in appearance. Its main ingredients are rice and black gram."  
documents = [doc_trump, doc_election, doc_putin, doc_soup, doc_noodles, doc_dosa]

In [17]:
# %pip install --user gensim


In [16]:

import gensim # upgrade gensim if you can't import softcossim 
from gensim.matutils import softcossim  
from gensim import corpora 
import gensim.downloader as api 
from gensim.utils import simple_preprocess 
print(gensim.__version__) #> '3.6.0'  
# Download the FastText model 
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300') 


3.8.0
[==================================================] 100.0% 958.5/958.4MB downloaded


In [18]:
import gensim 
# upgrade gensim if you can't import softcossim 
from gensim.matutils import softcossim  
from gensim import corpora 
import gensim.downloader as api 
from gensim.utils import simple_preprocess 
print(gensim.__version__) #> '3.6.0'  
# Download the FastText model fasttext_model300 = api.load('fasttext-wiki-news-subwords-300') 

3.8.0


In [21]:
# Prepare a dictionary and a corpus. 
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])  
# Prepare the similarity matrix 
similarity_matrix=fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)  
# Convert the sentences into bag-of-words vectors. 
sent_1 = dictionary.doc2bow(simple_preprocess(doc_trump)) 
sent_2 = dictionary.doc2bow(simple_preprocess(doc_election)) 
sent_3 = dictionary.doc2bow(simple_preprocess(doc_putin)) 
sent_4 = dictionary.doc2bow(simple_preprocess(doc_soup)) 
sent_5 = dictionary.doc2bow(simple_preprocess(doc_noodles)) 
sent_6 = dictionary.doc2bow(simple_preprocess(doc_dosa))  
sentences = [sent_1, sent_2, sent_3, sent_4, sent_5, sent_6]

/home/ibab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity_matrix` (Method will be removed in 4.0.0, use gensim.models.keyedvectors.WordEmbeddingSimilarityIndex instead).
  after removing the cwd from sys.path.


If you want the soft cosine similarity of 2 documents, you can just call the softcossim() function


In [22]:
# Compute soft cosine similarity 
print(softcossim(sent_1, sent_2, similarity_matrix)) 

0.525585133571468


/home/ibab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  


But, I want to compare the soft cosines for all documents against each other. So, create the soft cosine similarity matrix

In [26]:
import numpy as np 
import pandas as pd  
def create_soft_cossim_matrix(sentences):     
    len_array = np.arange(len(sentences))     
    xx, yy = np.meshgrid(len_array, len_array)     
    cossim_mat = pd.DataFrame([[round(softcossim(sentences[i],sentences[j], similarity _matrix)
,2) for i, j in zip(x,y)] for y, x in zip(xx, yy)])     
    return cossim_mat  
soft_cosine_similarity_matrix(sentences) 

SyntaxError: invalid syntax (<ipython-input-26-f77c0e0a3c87>, line 6)

In [29]:
import numpy as np
import pandas as pd

def create_soft_cossim_matrix(sentences):
    len_array = np.arange(len(sentences))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences[i],sentences[j], similarity_matrix) ,2) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

create_soft_cossim_matrix(sentences)

/home/ibab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  import sys
/home/ibab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  import sys
/home/ibab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  import sys
/home/ibab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product ins

,0,1,2,3,4,5
0,1.00,0.53,0.57,0.28,0.32,0.39
1,0.53,1.00,0.52,0.27,0.34,0.47
2,0.57,0.52,1.00,0.20,0.26,0.36
3,0.28,0.27,0.20,1.00,0.49,0.38
4,0.32,0.34,0.26,0.49,1.00,0.55
5,0.39,0.47,0.36,0.38,0.55,1.00


As one might expect, the similarity scores amongst similar documents are higher 

All copyrights are reserved